In [1]:
import pandas as pd
import numpy as np
from gplearn.genetic import SymbolicRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt
import re

# CSV 파일에서 데이터 로드
file_path = '20240806_correlation_copy.csv'
df = pd.read_csv(file_path)

In [2]:
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

In [23]:
y_min, y_max = df['adiabatic_S1-S0 (kcal/mol)'].min(), df['adiabatic_S1-S0 (kcal/mol)'].max()

X = normalized_df[["homo (eV)", "lumo (eV)", "vertical_excitation_energy (eV)", "dipole_moment_norm_S0 (D)",
    "dipole_moment_norm_S1 (D)", "dipole_moment_norm_T1 (D)", "0-0_T1 (eV)",
    "adiabatic_T1-S0 (kcal/mol)", "reduction_S0 (kcal/mol)",
    "oxidation_S0 (kcal/mol)", "lumo-homo (eV)"]]
y = normalized_df['adiabatic_S1-S0 (kcal/mol)']

est = SymbolicRegressor(population_size=5000,
                        generations=50,
                        stopping_criteria=0.01,
                        function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg', 'inv'],
                        metric='mean absolute error',
                        parsimony_coefficient=0.01,
                        verbose=1,
                        random_state=0)

est.fit(X, y)

# 수식 출력
if hasattr(est, '_program'):
    print(est._program)
else:
    print("The model does not have a _program attribute.")

y_pred_normalized = est.predict(X)
y_pred = y_pred_normalized * (y_max - y_min) + y_min
y_actual = y * (y_max - y_min) + y_min

r2 = r2_score(y_actual, y_pred)
mae = MAE(y_actual, y_pred)
rmse = np.sqrt(MSE(y_actual, y_pred))

plt.figure()
plt.plot([y_min, y_max], [y_min, y_max], 'k-', lw=2)
plt.scatter(y_actual, y_pred, color = 'red', alpha=0.5)
plt.xlabel(f'Actual values of adiabatic_S1-S0 (kcal/mol)')
plt.ylabel(f'Predicted values of adiabatic_S1-S0 (kcal/mol)')
plt.title('Parity Plot of adiabatic_S1-S0 (kcal/mol)')

# R^2 값 그래프에 추가
plt.text(x=0.05, y=0.95, s=f'$R^2 = {r2:.4f}$', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.90, s=f'MAE = {mae:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.85, s=f'RMSE = {rmse:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)

# 그래프 이미지 파일로 저장
plt.savefig('GPlearn Adiabatic1.png')
plt.close()


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.17           7.8997        5         0.058332              N/A      2.33m
   1     3.47         0.387655        4        0.0567614              N/A      2.42m
   2     1.90         0.159303        6        0.0562854              N/A      2.20m
   3     2.02         0.153246        5        0.0543821              N/A      2.25m
   4     4.00         0.188877        5        0.0543821              N/A      2.12m
   5     4.03          0.13123        4        0.0567614              N/A      2.19m
   6     4.02         0.121969        4        0.0567614              N/A      2.06m
   7     4.00         0.127847        4        0.0567614              N/A      2.03m
   8     3.98         0.121131        5        0.0558059              N/A  

In [24]:
y_min, y_max = df['adiabatic_S1-S0 (kcal/mol)'].min(), df['adiabatic_S1-S0 (kcal/mol)'].max()

X = normalized_df[["homo (eV)", "lumo (eV)", "vertical_excitation_energy (eV)", "dipole_moment_norm_S0 (D)",
    "dipole_moment_norm_S1 (D)", "dipole_moment_norm_T1 (D)", "0-0_T1 (eV)",
    "adiabatic_T1-S0 (kcal/mol)", "reduction_S0 (kcal/mol)",
    "oxidation_S0 (kcal/mol)", "lumo-homo (eV)"]]
y = normalized_df['adiabatic_S1-S0 (kcal/mol)']

est = SymbolicRegressor(population_size=5000,
                           generations=50, stopping_criteria=0.01,
                           function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg', 'inv'],
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)

est.fit(X, y)

# 수식 출력
print(est._program)

y_pred_normalized = est.predict(X)
y_pred = y_pred_normalized * (y_max - y_min) + y_min
y_actual = y * (y_max - y_min) + y_min

r2 = r2_score(y_actual, y_pred)
mae = MAE(y_actual, y_pred)
rmse = np.sqrt(MSE(y_actual, y_pred))

plt.figure()
plt.plot([y_min, y_max], [y_min, y_max], 'k-', lw=2)
plt.scatter(y_actual, y_pred, color = 'red', alpha=0.5)
plt.xlabel(f'Actual values of adiabatic_S1-S0 (kcal/mol)')
plt.ylabel(f'Predicted values of adiabatic_S1-S0 (kcal/mol)')
plt.title('Parity Plot of adiabatic_S1-S0 (kcal/mol)')

# R^2 값 그래프에 추가
plt.text(x=0.05, y=0.95, s=f'$R^2 = {r2:.4f}$', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.90, s=f'MAE = {mae:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.85, s=f'RMSE = {rmse:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)

# 그래프 이미지 파일로 저장
plt.savefig('GPlearn Adiabatic2.png')
plt.close()


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.17          8.04067        5        0.0584091        0.0576412      2.82m
   1     3.57         0.440483        4        0.0573004        0.0519303      2.58m
   2     2.11         0.306612        6        0.0558561        0.0601335      2.43m
   3     1.82         0.409551        4        0.0552634        0.0701896      2.34m
   4     3.28         0.216659        4        0.0556511        0.0667144      2.34m
   5     4.12         0.233202        4         0.048982        0.0481853      2.32m
   6     4.06           0.2534        4        0.0482219        0.0549982      2.33m
   7     3.91         0.216227        4        0.0479351        0.0575695      2.25m
   8     3.21         0.235963        4        0.0478125        0.0537819  

In [26]:
# 결측값 제거
valid_data = df[['adiabatic_S1-S0 (kcal/mol)', 'vertical_excitation_energy (eV)', 'dipole_moment_norm_S1 (D)', 'dipole_moment_norm_T1 (D)', '0-0_T1 (eV)', 'reduction_S0 (kcal/mol)']].dropna()

# 독립 변수와 종속 변수 업데이트 (결측값 제거된 데이터 사용)
X = valid_data[['vertical_excitation_energy (eV)', 'dipole_moment_norm_S1 (D)', 'dipole_moment_norm_T1 (D)', '0-0_T1 (eV)', 'reduction_S0 (kcal/mol)']]
y = valid_data['adiabatic_S1-S0 (kcal/mol)']

std_dev = y.std(ddof=0)
std_dev2 = y.std(ddof=0)*23.06
print(std_dev)

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

r2 = r2_score(y, y_pred)
mae = MAE(y, y_pred)
mae2 = MAE(y, y_pred)*23.06
rmse = np.sqrt(MSE(y, y_pred))
rmse2 = np.sqrt(MSE(y, y_pred))*23.06

# 실제 값과 예측치 저장
result_df = pd.DataFrame({
    'Real': y,
    'Predict': y_pred
})

# 파일 이름에서 유효하지 않은 문자 제거
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

csv_filename = sanitize_filename('Multiple Regression Adiabatic.csv')
result_df.to_csv(csv_filename, index=False)


# 그래프 생성
plt.figure()
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k-', lw=2)
plt.scatter(y, y_pred, color = 'red', alpha=0.5)
plt.xlabel(f'Actual values of adiabatic_S1-S0 (kcal/mol)')
plt.ylabel(f'Predicted values of adiabatic_S1-S0 (kcal/mol)')
plt.title('Parity Plot of adiabatic_S1-S0 (kcal/mol)')

# R^2 값 그래프에 추가
plt.text(x=0.05, y=0.95, s=f'$R^2 = {r2:.4f}$', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.90, s=f'MAE = {mae:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.85, s=f'RMSE = {rmse:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)

# 그래프 이미지 파일로 저장
img_filename = sanitize_filename('Multiple Regression Adiabatic.png')
plt.savefig(img_filename)
plt.close()


9.49494266443264


In [23]:
# 결측값 제거
valid_data = df[['reduction_S0 (kcal/mol)', 'homo (eV)', '0-0_S1 (eV)', 'adiabatic_T1-S0 (kcal/mol)']].dropna()

# 독립 변수와 종속 변수 업데이트 (결측값 제거된 데이터 사용)
y = valid_data['reduction_S0 (kcal/mol)']
X = valid_data[['homo (eV)', '0-0_S1 (eV)', 'adiabatic_T1-S0 (kcal/mol)']]

std_dev = y.std(ddof=0)
std_dev2 = y.std(ddof=0)*23.06
print(std_dev)

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

r2 = r2_score(y, y_pred)
mae = MAE(y, y_pred)
mae2 = MAE(y, y_pred)*23.06
rmse = np.sqrt(MSE(y, y_pred))
rmse2 = np.sqrt(MSE(y, y_pred))*23.06

# 실제 값과 예측치 저장
result_df = pd.DataFrame({
    'Real': y,
    'Predict': y_pred
})

# 파일 이름에서 유효하지 않은 문자 제거
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

csv_filename = sanitize_filename('Multiple Regression Reduction.csv')
result_df.to_csv(csv_filename, index=False)


# 그래프 생성
plt.figure()
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k-', lw=2)
plt.scatter(y, y_pred, color = 'red', alpha=0.5)
plt.xlabel(f'Actual values of reduction_S0 (kcal/mol)')
plt.ylabel(f'Predicted values of reduction_S0 (kcal/mol)')
plt.title('Parity Plot of reduction_S0 (kcal/mol)')

# R^2 값 그래프에 추가
plt.text(x=0.05, y=0.95, s=f'$R^2 = {r2:.4f}$', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.90, s=f'MAE = {mae:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.85, s=f'RMSE = {rmse:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)

# 그래프 이미지 파일로 저장
img_filename = sanitize_filename('Multiple Regression Reduction.png')
plt.savefig(img_filename)
plt.close()


6.325547264995044


In [24]:
# 결측값 제거
valid_data = df[['oxidation_S0 (kcal/mol)', 'lumo (eV)', 'dipole_moment_norm_S1 (D)', 'dipole_moment_norm_T1 (D)', '0-0_S1 (eV)', 'adiabatic_T1-S0 (kcal/mol)']].dropna()

# 독립 변수와 종속 변수 업데이트 (결측값 제거된 데이터 사용)
y = valid_data['oxidation_S0 (kcal/mol)']
X = valid_data[['lumo (eV)', 'dipole_moment_norm_S1 (D)', 'dipole_moment_norm_T1 (D)', '0-0_S1 (eV)', 'adiabatic_T1-S0 (kcal/mol)']]

std_dev = y.std(ddof=0)
std_dev2 = y.std(ddof=0)*23.06
print(std_dev)

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

r2 = r2_score(y, y_pred)
mae = MAE(y, y_pred)
mae2 = MAE(y, y_pred)*23.06
rmse = np.sqrt(MSE(y, y_pred))
rmse2 = np.sqrt(MSE(y, y_pred))*23.06

# 실제 값과 예측치 저장
result_df = pd.DataFrame({
    'Real': y,
    'Predict': y_pred
})

# 파일 이름에서 유효하지 않은 문자 제거
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

csv_filename = sanitize_filename('Multiple Regression Oxidation.csv')
result_df.to_csv(csv_filename, index=False)


# 그래프 생성
plt.figure()
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k-', lw=2)
plt.scatter(y, y_pred, color = 'red', alpha=0.5)
plt.xlabel(f'Actual values of oxidation_S0 (kcal/mol)')
plt.ylabel(f'Predicted values of oxidation_S0 (kcal/mol)')
plt.title('Parity Plot of oxidation_S0 (kcal/mol)')

# R^2 값 그래프에 추가
plt.text(x=0.05, y=0.95, s=f'$R^2 = {r2:.4f}$', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.90, s=f'MAE = {mae:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)
plt.text(x=0.05, y=0.85, s=f'RMSE = {rmse:.4f} (kcal/mol)', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)

# 그래프 이미지 파일로 저장
img_filename = sanitize_filename('Multiple Regression Oxidation.png')
plt.savefig(img_filename)
plt.close()


15.161931325309032


In [21]:
# 결측값 제거
valid_data = df[['dipole_moment_norm_S0 (D)']].dropna()

std_dev = valid_data.std(ddof=0)
std_dev2 = valid_data.std(ddof=0)*23.06
print(std_dev)

dipole_moment_norm_S0 (D)    3.89232
dtype: float64
